# SVM on all classes 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
from scipy.signal import spectrogram
from sklearn.model_selection import train_test_split

In [2]:
eeg_df = pd.read_csv("../data/2018-10-12-preston_walk_raw.csv", skiprows=15)

In [3]:
eeg_df.head()

,Time,LE,F4,C4,PO8,PO7,C3,F3,Trigger,Time_Offset,ADC_Status,ADC_Sequence,Event,Comments
0,0.0033,3642.0,-2536.8,3530.1,2459.4,296.7,3454.5,-1103.4,0,6,0,207,0,Start Data Acquisition
1,0.0067,3635.7,-2526.3,3531.0,2451.9,299.7,3459.6,-1098.9,0,12,0,208,0,NaN
2,0.0100,3632.7,-2535.9,3528.6,2441.4,302.1,3454.8,-1103.4,0,18,0,209,0,NaN
3,0.0133,3626.7,-2535.3,3524.1,2450.1,297.9,3455.1,-1097.1,0,24,0,210,0,NaN
4,0.0167,3626.4,-2527.8,3539.4,2469.0,289.8,3452.7,-1089.9,0,30,0,211,0,NaN


## Filter data

In [4]:
from mne.filter import filter_data

In [5]:
channels = eeg_df.columns[1:8] 

In [6]:
eeg_data = eeg_df[channels].values

In [7]:
filtered = filter_data(eeg_data.T, sfreq=300, l_freq=1, h_freq=40)

Setting up band-pass filter from 1 - 40 Hz
l_trans_bandwidth chosen to be 1.0 Hz
h_trans_bandwidth chosen to be 10.0 Hz
Filter length of 991 samples (3.303 sec) selected


In [8]:
eeg_df[channels] = filtered.T

## Epoch data

In [9]:
events = eeg_df.Event.nonzero()[0]

In [10]:
events

array([  1446,   2970,   7952,  47145,  78073, 135086])

In [11]:
walk_one_start = events[2] + 300
walk_one_stop = events[3] - 300
typing_start = events[3] + 300
typing_stop = events[4] - 300
talking_start = events[4] + 300  
talking_stop = events[5] - 300  
walk_two_start = events[5] + 300
walk_two_stop = len(eeg_df) - 9000

In [12]:
typing_data = eeg_df.loc[typing_start : typing_stop][channels]
talking_data = eeg_df.loc[talking_start : talking_stop][channels]
walk_one_data = eeg_df.loc[walk_one_start : walk_one_stop][channels]
walk_two_data = eeg_df.loc[walk_two_start : walk_two_stop][channels]

In [13]:
typing_data = typing_data[: len(typing_data) // 300 * 300]
talking_data = talking_data[: len(talking_data) // 300 * 300]
walk_one_data = walk_one_data[: len(walk_one_data) // 300 * 300]
walk_two_data = walk_two_data[: len(walk_two_data) // 300 * 300]
walk_data = np.concatenate((walk_one_data, walk_two_data), axis=0)

## Power, create data

In [14]:
data = np.concatenate((typing_data, talking_data, walk_data), axis=0) 

In [15]:
f, t, typing_spec = spectrogram(typing_data.T, fs=300, nperseg=600, noverlap=300)

In [16]:
typing_spec = typing_spec[:, :81, :].reshape(-1, typing_spec.shape[2]).T

In [17]:
f, t, talking_spec = spectrogram(talking_data.T, fs=300, nperseg=600, noverlap=300)
talking_spec = talking_spec[:, :81, :].reshape(-1, talking_spec.shape[2]).T

In [18]:
f, t, walk_spec = spectrogram(walk_data.T, fs=300, nperseg=600, noverlap=300)
walk_spec = walk_spec[:, :81, :].reshape(-1, walk_spec.shape[2]).T

In [19]:
X = np.concatenate((typing_spec, talking_spec, walk_spec), axis=0) 

In [20]:
y = np.concatenate((np.zeros(len(typing_spec)), np.ones(len(talking_spec)), np.full((len(walk_spec), ), 2)), axis=0)

In [21]:
# shuffle data to get train/test
args = np.arange(len(X))
np.random.shuffle(args)
X = X[args]
y = y[args]

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

## SVM

In [23]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

In [89]:
svc = SVC(kernel='poly', C=0.1, degree=1, verbose=True)

In [90]:
svc.fit(X_train, y_train)

[LibSVM]

SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=1, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=True)

In [91]:
svc.score(X_train, y_train)

0.9977973568281938

In [92]:
svc.score(X_test, y_test)

0.7843137254901961

## Neural Networks

In [133]:
from sklearn.neural_network import MLPClassifier

In [134]:
hids = [(100, ), (200, ), (250, ), (400, ), (500, ), (100, 50), (200, 100), (500, 100)]
alphas = [0.1, 0.01, 0.001, 0.0001]

In [135]:
grid = GridSearchCV(MLPClassifier(), param_grid={'hidden_layer_sizes':hids, 'alpha':alphas}, cv=KFold(n_splits=10), verbose=True)

In [136]:
grid.fit(X_train, y_train)

Fitting 10 folds for each of 32 candidates, totalling 320 fits


[Parallel(n_jobs=1)]: Done 320 out of 320 | elapsed:  2.7min finished


GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=False),
       error_score='raise',
       estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'hidden_layer_sizes': [(100,), (200,), (250,), (400,), (500,), (100, 50), (200, 100), (500, 100)], 'alpha': [0.1, 0.01, 0.001, 0.0001]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=True)

In [137]:
grid.best_estimator_

MLPClassifier(activation='relu', alpha=0.01, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(250,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [142]:
best_net = MLPClassifier(hidden_layer_sizes=(250, ), alpha=0.01) #grid.best_estimator_

In [143]:
best_net.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.01, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(250,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [144]:
best_net.score(X_train, y_train)

1.0

In [145]:
best_net.score(X_test, y_test)

0.8627450980392157